# 1. Установка инструментов для дальнейшего использования

In [ ]:
!pip install setuptools==65.5.0 "wheel<0.40.0"
!pip install gym[atari]==0.19.0
!pip install gym[accept-rom-license]
!pip install autorom

# 2. Импорт используемых в работе библиотек

In [3]:
import gym.wrappers
import numpy as np
from gym.wrappers import AtariPreprocessing, FrameStack

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

# 3. Создание среды Breakout. Обработка этой среды, получаю четыре последних кадры игры, чтобы понимать динамику игры. И делаю оттенка серого, чтобы моей модели было проще обучаться.

In [5]:
env = gym.make("BreakoutNoFrameskip-v4")
env = AtariPreprocessing(env, grayscale_newaxis=True, scale_obs=True)
env = FrameStack(env, 4)
num_actions = env.action_space.n

# 4. Архитектура модели. Использую три слоя сверточных нейронных сетей, и обыкновенный перцептрон.

In [6]:
class QNetwork(nn.Module):
    def __init__(self):
        super(QNetwork, self).__init__()
        self.conv1 = nn.Conv2d(4, 32, kernel_size=8, stride=4)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=4, stride=2)
        self.conv3 = nn.Conv2d(64, 64, kernel_size=3, stride=1)
        self.fc1 = nn.Linear(7 * 7 * 64, 652)
        self.fc2 = nn.Linear(652, num_actions)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = x.reshape(x.size(0), -1)  # Flatten
        x = F.relu(self.fc1(x))
        return self.fc2(x)

# 5. Выбор гипрепараметров

In [7]:
gamma = 0.99 # Коэффициент дисконтирования

# Выбор эпсилон, для жадной стратегии
epsilon = 1.0
epsilon_max_1 = 1.0
epsilon_min_1 = 0.2
epsilon_max_2 = epsilon_min_1
epsilon_min_2 = 0.1
epsilon_max_3 = epsilon_min_2
epsilon_min_3 = 0.02

epsilon_interval_1 = (epsilon_max_1 - epsilon_min_1)
epsilon_interval_2 = (epsilon_max_2 - epsilon_min_2)
epsilon_interval_3 = (epsilon_max_3 - epsilon_min_3)

# Количество фреймов для исследования
epsilon_greedy_frames = 1000000.0

# Количество фреймов, при котором будет совершаться рандомное действие
epsilon_random_frames = 50000

# Размер Replay Buffer'a для хранения состояния игры
max_memory_length = 80000

# Размер батча, который будет браться из Replay Buffer'a
batch_size = 32

# Максимальное кол-во шагов за эпизод
max_steps_per_episode = 10000

# Обучение будет происходить каждые 20 действой
update_after_action = 20

# Как часто будет обновляться целевая модель
update_target_network = 10000


# 6. Объявление модели агента и целевой модели, так же использую функцию потерь Хьюбера, и оптимизатор Адама.

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # Использую GPU
model = QNetwork().to(device)
# Target model
model_target = QNetwork().to(device)
# Using MSE loss for stability
loss_function = nn.SmoothL1Loss()
optimizer = optim.Adam(model.parameters(), lr=0.00025)

# 7. Инициализация Replay Buffer'a и необходоимых переменных

In [9]:
action_history = []
state_history = []
state_next_history = []
rewards_history = []
done_history = []

episode_reward_history = []
running_reward = 0
episode_count = 0
frame_count = 0

# 8. Цикл обучения

In [ ]:
while True:
    state = np.array(env.reset())
    state = state.reshape(84, 84, 4)
    episode_reward = 0

    for timestep in range(1, max_steps_per_episode):
        frame_count += 1
        
        # Эпсилон жадная стратегия
        if frame_count < epsilon_random_frames or epsilon > np.random.rand(1)[0]:
            # Выбираю рандомное действие
            action = np.random.choice(num_actions)
        else:
            # Предсказываю функцию полезности действия
            state_tensor = torch.tensor(state).unsqueeze(0)
            state_tensor = state_tensor.permute(0, 3, 1, 2)
            state_tensor= state_tensor.to(device)
            action_probs = model(state_tensor).to(device)
            # Выбираю лучшее действие
            action = torch.argmax(action_probs).item()

        # Уменьшаю эпсилон
        if frame_count < epsilon_greedy_frames:
            epsilon -= epsilon_interval_1 / epsilon_greedy_frames
            epsilon = max(epsilon, epsilon_min_1)

        if frame_count > epsilon_greedy_frames and frame_count < 2 * epsilon_greedy_frames:
            epsilon -= epsilon_interval_2 / epsilon_greedy_frames
            epsilon = max(epsilon, epsilon_min_2)

        if frame_count > 2 * epsilon_greedy_frames:
            epsilon -= epsilon_interval_3 / epsilon_greedy_frames
            epsilon = max(epsilon, epsilon_min_3)

        # Совершаю действие
        state_next, reward, done, _ = env.step(action)
        state_next = np.array(state_next)
        state_next = state_next.reshape(84, 84, 4)
        
        episode_reward += reward

        # Сохраняю собранные данные в буффер
        action_history.append(action)
        state_history.append(state)
        state_next_history.append(state_next)
        done_history.append(done)
        rewards_history.append(reward)
        state = state_next
  
        # Обновляю сеть каждые 20 фреймов и беру данные из буфера размером 32
        if frame_count % update_after_action == 0 and len(done_history) > batch_size:

            # Беру рандомные индексы из буффера размером 32
            indices = np.random.choice(range(len(done_history)), size=batch_size)
          
        
            # Теперь беру данные из буффера по этим индексам
            state_sample = torch.tensor(np.array([state_history[i] for i in indices])).float()
            state_next_sample = torch.tensor(np.array([state_next_history[i] for i in indices])).float()
            rewards_sample = torch.tensor(np.array([rewards_history[i] for i in indices])).float()
            action_sample = torch.tensor(np.array([action_history[i] for i in indices])).long()
            done_sample = torch.tensor([float(done_history[i]) for i in indices])

            state_next_sample = state_next_sample.permute(0, 3, 1, 2)
            state_sample = state_sample.permute(0, 3, 1, 2)
            
            state_sample = state_sample.to(device)
            action_sample = action_sample.to(device)
            rewards_sample = rewards_sample.to(device)
            state_next_sample = state_next_sample.to(device)
            done_sample = done_sample.to(device)

            # Создаю обновленные значения Q для выбранных будущих состояний
            # Использую целевую функцию для стабильности
            future_rewards = model_target(state_next_sample)
            # Q_value = reward + discount factor * expected future reward
            updated_q_values = rewards_sample + gamma * torch.max(future_rewards, dim=1).values

            
            updated_q_values = updated_q_values * (1 - done_sample) - done_sample

            # Создаю маски, чтобы рассчитать потери для обнавленных масок Q значений
            masks = F.one_hot(action_sample, num_actions)

            # Вычисляем значение
            q_values = model(state_sample)

            # Применяем маски к значениям Q, чтобы получить значение Q для предпринятого действия
            q_action = torch.sum(q_values * masks, dim=1)

            # Считаю потерю между новой Q-функцие и старой
            loss = loss_function(q_action, updated_q_values)

            # Обратное распространение
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        if frame_count % update_target_network == 0:
            # Обновляю веса целевой функции
            model_target.load_state_dict(model.state_dict())
            # Промежуточные выводы
            print(f"running reward: {running_reward:.2f} at episode {episode_count}, frame count {frame_count}, epsilon {epsilon:.3f}, loss {loss.item():.5f}")

        # Очищаем буффер чтобы он не переполнялся
        if len(rewards_history) > max_memory_length:
            del rewards_history[:1]
            del state_history[:1]
            del state_next_history[:1]
            del action_history[:1]
            del done_history[:1]

        if done:
            break
    
    # Каждые 500 фрейвом высчитываю среднее вознаграждение
    episode_reward_history.append(episode_reward)
    if len(episode_reward_history) > 500:
        del episode_reward_history[:1]
    running_reward = np.mean(episode_reward_history)

    episode_count += 1

    # Завершаю игру если срднее вознаграждение больше 100
    if running_reward > 100:
      print("Solved at episode {}!".format(episode_count))
      break

In [12]:
torch.save(model, '/kaggle/working/model_breakout.pt')

# Делаем видео игры

In [ ]:
torch.save(model, 'new_model_breakout_150_reward.pt')
torch.save(model_target, 'new__target_model_breakout_150_reward.pt')

In [ ]:
def make_env():
    env = gym.make("BreakoutNoFrameskip-v4")
    env = AtariPreprocessing(env, grayscale_newaxis=True, scale_obs=True)
    env = FrameStack(env, 4)
    return env

In [ ]:
env = make_env()
env = gym.wrappers.Monitor(env, "./vid1", force=True)

observation = np.array(env.reset())
observation = observation.reshape(84, 84, 4)

info = 0
reward_window = []

hits = []
bltd = 108 #общее кол-во блоков

for i_episode in range(1):
    reward_window=[] 
    epsilon = 0  
    for t in range(5000):
        if epsilon > np.random.rand(1)[0]:
            # Выбираем рандомное действие
            action = np.random.choice(4)
        else:
            # Предсказываю функцию полезности действия
            state_tensor = torch.tensor(observation).unsqueeze(0)
            state_tensor = state_tensor.permute(0, 3, 1, 2)
            state_tensor= state_tensor.to(device)
            action_probs = model(state_tensor).to(device)
            # Выбираю лучшее действие
            action = torch.argmax(action_probs).item()
        observation, reward, done, info = env.step(action)
        observation = np.array(observation)

        observation = observation.reshape(84, 84, 4)
        hits.append(reward)
        reward_window.append(reward)
        if len(reward_window) > 500:
          del reward_window[:1] 
        if len(reward_window) == 500 and np.sum(reward_window) == 0:
          print(reward_window)
          epsilon = 0.01
        else:
          epsilon = 0.0001

        
        if done:
            print("Потерял одну жизнь за {} время".format(t+1))
            print(info)
            
            bltd = bltd-np.sum(hits)
            hits = []
            print("Было сломано блоков ", bltd)
            if info['ale.lives'] == 0:
              break

            env.reset()
env.close()